## Load the FASTA file from the GtRNAdb server

The urllib is used here, but it will be later changed to use keras's resource caching utility.

In [29]:
from keras.utils.data_utils import get_file

fasta_path = get_file('sacCer3-tRNAs.fa', origin='http://gtrnadb.ucsc.edu/genomes/eukaryota/Scere3/sacCer3-tRNAs.fa')

In [30]:
from Bio.SeqIO.FastaIO import SimpleFastaParser

# generator is iterable
sequence_generator = SimpleFastaParser(open(fasta_path))
    
fasta_sequences = [(seq) for (title, seq) in sequence_generator]

In [31]:
# fasta_sequences holds the original sequences from the file
fasta_sequences[:5]

['GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGtCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGtCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGtCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGtCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGtCTCCGGTTCGATTCCGGACTCGTCCA']

In [24]:
sequences = [(seq.upper()) for (seq) in fasta_sequences]

In [33]:
# sequences holds the uppercase sequences from the file
sequences[:5]

['GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGTCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGTCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGTCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGTCTCCGGTTCGATTCCGGACTCGTCCA',
 'GGGCGTGTGGCGTAGTCGGTAGCGCGCTCCCTTAGCATGGGAGAGGTCTCCGGTTCGATTCCGGACTCGTCCA']

For now, the sequences are all upper case to make the neural network more effective at determining sequence.

## Vectorizing the sequences

In [37]:
# these should be later changed to include lower case as well
# chars = ['A', 'C', 'G', 'T']
chars = list(set(''.join(fasta_sequences)))

char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

['g', 'G', 't', 'c', 'T', 'C', 'a', 'A']


In [39]:
# how much we tell the rnn before it has to guess
test_length = 25
step = 3

# list of all the tests, semiredundant
test_sequences = []
next_bases = []

# load in training cases
# for seq in sequences:
for seq in fasta_sequences:
    for i in range(0, len(seq) - test_length, step):
        test_sequences.append(seq[i: i + test_length])
        next_bases.append(seq[i + test_length])

In [40]:
import numpy as np

# turn into vectors
# first dimension - each test case
# second dimension - number of hints to give
# third dimension - boolean representation for chars
X = np.zeros((len(test_sequences), test_length, len(chars)), dtype=np.bool)
y = np.zeros((len(next_bases), len(chars)), dtype=np.bool)

for i, seq in enumerate(test_sequences):
    for t, char in enumerate(seq):
        X[i, t, char2index[char]] = 1
    y[i, char2index[next_bases[i]]] = 1

In [41]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM

print('Build the single LSTM model...')
model = Sequential()
model.add(LSTM(128, input_shape=(test_length, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build the single LSTM model...


In [42]:
from keras.optimizers import RMSprop
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [88]:
def splice_introns(rna_sequence):
    spliced_rna_sequence = ''.join([(nt) for (nt) in rna_sequence if nt.isupper()])
    return spliced_rna_sequence

In [79]:
def transcribe(dna_sequence):
    rna_sequence = Seq(dna_sequence, generic_dna).transcribe()
    return rna_sequence

In [86]:
def create_barnacle(filename, rna_sequence):
    spliced_rna_sequence = splice_introns(rna_sequence)
    rna_model = Barnacle(spliced_rna_sequence)
    rna_model.sample()
    rna_model.save_structure(filename)

In [87]:
from Barnacle import Barnacle

from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

import os
os.makedirs('output', exist_ok=True)


for i in range(30):
    print('round %i' % i)
    model.fit(X, y, batch_size=128, epochs=1)

    pred = model.predict(X[0:1])
    pred_index = np.argmax(pred)
    pred_char = index2char[pred_index]
    print(pred_char, flush=True)

    generated = ''
    sentence = sequences[0][0: test_length]
    generated += sentence

    print(generated)

    for j in range(73 - test_length):
        x = np.zeros((1, test_length, len(chars)), dtype=np.bool)
        for t, char in enumerate(sentence):
            x[0, t, char2index[char]] = 1
        pred = model.predict(x)
        pred_index = np.argmax(pred)
        pred_char = index2char[pred_index]

        generated += pred_char
        sentence = sentence[1:] + pred_char

    print(sequences[0])
    print(generated, flush=True)
    
    rna_sequence = transcribe(generated)
    create_barnacle("output/output%i.pdb" % i, rna_sequence)

round 0
Epoch 1/1
5109/5109 [==============================] - 3s - loss: 0.0608     